*Copyright (C) 2021 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# Connect processes

This tutorial gives an introduction in how to connect _Processes_ to build a network of asynchronously operating and interacting _Processes_.

## Recommended tutorials before starting:

- [Installing Lava](./tutorial01_installing_lava.ipynb "Tutorial on Installing Lava")
- [Processes](./tutorial02_processes.ipynb "Tutorial on Processes")
- [ProcessModel](./tutorial03_process_models.ipynb "Tutorial on ProcessModels")
- [Execution](./tutorial04_execution.ipynb "Tutorial on Executing Processes")

    
## Building a network of _Processes_

_Processes_ are the main building blocks of Lava. Each _Process_ can exercise different computations and usually depends on some input data and/or creates output data. Transfering I/O data between _Processes_ is a key element of Lava. A _Process_ can have various input and output _Ports_ which are then connected via channels to corresponding _Ports_ of another _Process_. This allows to build networks of asynchronously operating and interacting _Processes_.

## Create a connection

The objective is to connect _Process_ _P1_ with _Process_ _P2_. _P1_ has an output _Port_ _OutPort_ called _out_ and _P2_ has an input port _InPort_ called _inp_. Data from _P1_ provided to the _Port_ _out_ should be transfered to _P2_ and received from _Port_ _inp_.

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig01.png" width="400"  />


In [ ]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.ports.ports import InPort, OutPort

As first step we define the _Processes_ _P1_ and _P2_ with their respective _Ports_ _out_ and _inp_.

In [ ]:
# minimal process with an OutPort
class P1(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.out = OutPort(shape=shape)


# minimal process with an InPort
class P2(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.inp = InPort(shape=shape)

_Process_ _P1_ and _P2_ require a corresponding _ProcessModel_, which implements their _Ports_ and a simple run configuration for sending and receiving data.

In the _ProcessModels_ python code should be exectued on a CPU. The input and output _Port_ should be able to receive/send a vector of integers and print the transferred data.

So the _ProcessModel_ inherits form _AbstractPyProcessModel_ in order to execute python code and the configured _ComputeResource_ is a CPU. A _LavaPyType_ is used for the _Ports_. The _LavaPyType_ specifies the expected data format for the _Port_. A dense vector of type integer is chosen with the parameters _PyOutPort.VEC_DENSE_ and _int_. The _Ports_ can be used to send and receive data by calling _send_ or _recv_. The sent and received data is afterwards printed out.

In [ ]:
import numpy as np
from lava.magma.core.model.py.model import AbstractPyProcessModel
from lava.magma.core.decorator import implements, requires
from lava.magma.core.resources import CPU
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.model.py.ports import PyInPort, PyOutPort

In [ ]:
# A minimal PyProcModel implementing P1
@implements(proc=P1)
@requires(CPU)
class PyProcModelA(AbstractPyProcessModel):
    out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, int)

    def run(self):
        data = np.array([1, 2])
        self.out.send(data)
        print("Sent output data of P1: ", str(data))


# A minimal PyProcModel implementing P2
@implements(proc=P2)
@requires(CPU)
class PyProcModelB(AbstractPyProcessModel):
    inp: PyInPort = LavaPyType(PyInPort.VEC_DENSE, int)

    def run(self):
        in_data = self.inp.recv()
        print("Received input data for P2: ", str(in_data))

Next the processes _P1_ and _P2_ are instantiated and the output _Port_ _out_ from _Process_ _P1_ is connected with the input _Port_ _inp_ of _Process_ _P2_.

In [ ]:
sender = P1()
recv = P2()

# Connecting output port to an input port
sender.out.connect(recv.inp)

sender = P1()
recv = P2()

# ... or connecting an input port from an output port
recv.inp.connect_from(sender.out)

Calling `run()` on either of these _Processes_ will first call the _Compiler_. During compilation the specified connection is setup by creating a channel between _P1_ and _P2_. Now data can be transfered during execution as seen by the output print statements.

In [ ]:
from lava.magma.core.run_configs import RunConfig
from lava.magma.core.run_conditions import RunSteps

In [ ]:
class MyRunCfg(RunConfig):
    def select(self, proc, proc_models):
        return proc_models[0]

In [ ]:

sender.run(RunSteps(num_steps=42, blocking=False), MyRunCfg())
sender.stop()

## Possible connections
This first example was very simple. In principle _Processes_ can have multiple input and output _Ports_, which can be freely connected with each other. Also _Processes_ which execute on different devices can be connected in the same way.

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig02.png" width="400"  />

#### There are some things to consider though:
- _InPorts_ cannot connect to _OutPorts_
- Shape and datatype of connect _Ports_ must match
- An _InPort_ might get data from multiple _OutPorts_ - default behavior is a summation of the incoming data
- An _OutPort_ might send data to multiple _InPorts_ - all _InPorts_ receive the same data


## Connect multiple _InPorts_ from a single _OutPort_

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig04.png" width="400"  />

In [ ]:
sender = P1()
recv1 = P2()
recv2 = P2()
recv3 = P2()

# An OutPort can connect to multiple InPorts
# Either at once...
sender.out.connect([recv1.inp, recv2.inp, recv3.inp])

sender = P1()
recv1 = P2()
recv2 = P2()
recv3 = P2()

# ... or consecutively
sender.out.connect(recv1.inp)
sender.out.connect(recv2.inp)
sender.out.connect(recv3.inp)

In [ ]:
# TODO: run it

## Connecting multiple _InPorts_ to a single _OutPort_

If multiple input _Ports_ connect to the same output _Port_, the default behavior is that the data from each input _Port_ is added up at the output _Port_. Another reduce operation would be multiplication.

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig05.png" width="400"  />

In [ ]:
sender1 = P1()
sender2 = P1()
sender3 = P1()
recv = P2()

# An InPort can connect to multiple OutPorts
# Either at once...
recv.inp.connect_from([sender1.out, sender2.out, sender3.out])

sender1 = P1()
sender2 = P1()
sender3 = P1()
recv = P2()

# ... or consecutively
sender1.out.connect(recv.inp)
sender2.out.connect(recv.inp)
sender3.out.connect(recv.inp)

In [ ]:
# TODO: show run

## Connecting _Ports_ with different shapes

It cannot be assumed that every _Process_ sends or receives data in the same shape. There might be _Processes_ from different users which work with different vector shapes for various reasons. Still, it is possible to connect such _Processes_ using the reshape feature of _Ports_.

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig03.png" width="400"  />

First, the _Processes_ _P1_ and _P2_ are instantiated with new shapes (1, 6) and (2, 3). The output port of _P1_ can still be conncected with the input port of _P2_, if appropriate reshape is used. The total number of elements needs to be the same for both ports.

In [ ]:
sender = P1(shape=(1,6))
recv = P2(shape=(2, 3))

# Using reshape(..), ports with different shape can be connected as
# long as total number of elements does not change
sender.out.reshape((2, 3)).connect(recv.inp)

In [ ]:
# TODO: show run

## Concatenation of _Ports_

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig06.png" width="400"  />

The _Processes_ _P1_ and _P2_ are instantiated with new shapes (1, 2) and (3, 2). Now the output ports of the sending processes are concatenated before connected to the receiving port.

In [ ]:
sender1 = P1(shape=(1, 2))
sender2 = P1(shape=(1, 2))
sender3 = P1(shape=(1, 2))
recv = P2(shape=(3, 2))

# concat_with(..) concatenates calling port (sender1.out) with other ports
# (sender2.out, sender3.out) along given axis
cp = sender1.out.concat_with([sender2.out, sender3.out], axis=0)

 # The return value is a virtual ConcatPort which can be connected to the input port
cp.connect(recv.inp)

In [ ]:
# TODO: run

## How to learn more?

Learn how to implement and compose the behavior of a process using other processes the [next tutorial on hierarchical Processes](./tutorial06_hierarchical_processes.ipynb "Tutorial on Hierarchical Processes").

If you want to find out more about connecting processes, have a look at the [Lava documentation](https://lava-nc.org/ "Lava Documentation") or dive into the [source code](https://github.com/lava-nc/lava/tree/main/lava/magma/core/process/ports/ports.py "Port Source Code").

To receive regular updates on the latest developments and releases of the Lava Software Framework please subscribe to the [INRC newsletter](http://eepurl.com/hJCyhb "INRC Newsletter").